In [1]:

secondtable = []
count = 0
for i in range(5):
    secondtable.append("<tr>")
    
    for j in range(3):
        secondtable.append("""
         <td align="center" valign="center">
            <img src="traj%i.png"  />   
            </td>"""%count)
        count += 1
    secondtable.append("</tr>")
secondtable = "\n".join(secondtable)
template = """
<!doctype html>
<head>
    <link rel="stylesheet" type="text/css" href="style.css">
</head>
<html>
    <body>
      <h1> %s </h1>
       <table>
        <tr>

            <td align="center" valign="center">
            
                <br />
                Brownian V separation 
                <br /> 
                without noise
                <br />
            <img src="fig1.png" alt="description here" />
           
            </td>
            
            <td align="center" valign="center">
            
                <br />
                Brownian V separation 
                <br /> 
                with 0.25 noise
                <br/>
             <img src="fig2.png" alt="description here" />

            </td>
      

            <td align="center" valign="center">

             Brownian Brownian separation
             <br/>
            <img src="fig3.png" alt="description here" />
            
            </td>

        </tr>
        </table>
        
        %s
    </body>
</html>"""


templatecss="""
ext_container {
    position: absolute;
    top: 50px;
}

#fig1_container {
    position: relative;
    float: left;
}

#fig2_container {
    position: relative;
    float: left;
    left: 100px;
}

#fig3_container {
    position: absolute;
    left: 50px
    /*float: left; */
}

#fig4_container {
    position: absolute;
    left: 500px;
}

#row2 {
    position: relative;
    top: 300px;
}

img {
    max-width: 300px;
    max-height: 300px;
}"""

with open("style.css","w") as f:
    f.writelines(templatecss)

In [35]:

from scipy.io import loadmat
from prePostTools import clean_initial_trajectory,clean,put_back_nan,M0,M1,traj_to_dist
from Tools import plot_by_class,plot_label
from scitool.propertie import Propertie
import numpy as np
sub = True
ndim=2
from pylab import *
#graph.load_weights("saved_weights/three_layer_trained_on_no_sub_anisentropy0_diffsigma1_delta_sigma_directed3")
#graph.load_weights("cutter_2_6_diff_size_50")
def create_images_report(graph,v=1):
    def process_one_file(filetoopen,graph,sm=0):
        trajs = []
        M = loadmat(filetoopen)


        pred_RNNs = []
        pred_RNN_cats = []

        M["analysisInfo"] = { "tool": "RNN",
                          "mu_emit_unit":"pixel",
                             "mu_real_unit":"pixel",
                          "sigma_emit_unit":"pixel",
                           "D_emit_unit":"Assuming your unit for the piel size was nanometer, in mu^2/s"}

        if sub:
             M["analysisInfo"]["PrM_labels"] = np.array(M1,dtype=np.object)
        else:
             M["analysisInfo"]["PrM_labels"] = np.array(M0,dtype=np.object)

        px = M["movieInfo"]["pixelSize"][0][0][0][0] / 1000. #In micro meter
        fr = M["movieInfo"]["frameRate"][0][0][0][0]


        M["results"] = []
        #NewField = ["ML_states","PrM","steps",'ML_params']

        #Copy old fields
        #M["tracksProc"] #= {field:M["tracksProc"][field][0] for field in M["tracksProc"].dtype.names}

        #print M["tracksProc"]["pos"].shape
        #Add new fields
        #tmp = {}
        #for field in NewField:
        #    M["tracksProc"][field] = np.empty(len(M["tracksProc"]["pos"]), dtype=np.object)

        ntraj = len(M["tracksProc"][0]["pos"])


        for itraj,traj0 in enumerate(M["tracksProc"][0]["pos"]):

            #print traj
            #traj0 = traj
            #traj0 = traj0[-00:]
            #print len(traj0)
            #print traj0.shape
            #print traj0.shape
            #if len(traj0) > 400:
            #    traj0 = traj0[100:,:]
            traj,alligned_traj,normed,zeros,nans,added0 = clean_initial_trajectory(traj0,v=v)
            if sm != 0:

                """
                traj[::,0] = Propertie(traj[::,0]).smooth(sm)
                traj[::,1] = Propertie(traj[::,1]).smooth(sm)

                alligned_traj,normed,alpha,_ = traj_to_dist(traj,ndim=2)
                """


                real_traj1 = np.array([Propertie(traj[::,0]).smooth(2),
                                       Propertie(traj[::,1]).smooth(2)])

                alligned_traj1,normed1,alpha1,_ = traj_to_dist(real_traj1.T,ndim=ndim)
                real_traj2 = np.array([Propertie(traj[::,0]).smooth(10),
                                       Propertie(traj[::,1]).smooth(10)])
                alligned_traj2,normed2,alpha2,_ = traj_to_dist(real_traj2.T,ndim=ndim)

                normed = np.concatenate((normed,normed1,normed2),axis=1)


            trajs.append(traj)
            #print normed.shape
            #pred0 = graph9.predict({"input1":normed[newaxis,::,::]})


            pred0 = graph.predict({"input1":normed[np.newaxis,::,::]},  batch_size=1)

            pred_RNN = pred0["output"]
            pred_RNN_cat = pred0["category"]

            if len(pred_RNN_cat.shape) == 3:
                pred_RNN_cat = pred_RNN_cat[0]

            #Inverse these 
            if not sub:
                #print pred_RNN_cat.shape

                pred_RNN_cat = pred_RNN_cat[::,:len(M0)]


            pred_RNN = pred_RNN[0]
            pred_RNN_cat = pred_RNN_cat

            if added0:
                zeros.pop(-1)
                traj = traj[:-1,::]
                pred_RNN = pred_RNN[:-1]

            pred_RNNs.append(pred_RNN)
            pred_RNN_cats.append(pred_RNN_cat)



            cat =  pred_RNN

            fight = False
            cat = clean(cat,np.argmax(pred_RNN_cat),fight=fight,sub=sub,append_steady=False)

            fight = True
            if fight:
                pred_RNN_cat[0,8:10] = pred_RNN_cat[0,8:10][::-1]

            cat = cat.tolist()

            cat = put_back_nan(cat,zeros,nans)

            assert(len(cat) == len(traj0)-1)






            M["results"].append( [traj0,cat,pred_RNN_cat])


            #print cat

        #M["tracksProc"] = np.array(M["tracksProc"],dtype=)

        return M


    In = [["Inspected/Inspected/v3_crop-1_MinFr30.mat",0],
          ["Inspected/Inspected/v3_crop-2_MinFr30.mat",0],
          ["Inspected/Inspected/v3_crop-2_MinFr30.mat",1],
          ["Inspected/Inspected/v3_crop-2_MinFr30.mat",2],
          ["Inspected/Inspected/v3_crop-3_MinFr30.mat",0],
          ["Inspected/Inspected/v3_crop-3_MinFr30.mat",1],
          ["Inspected/Inspected/v8_crop-1_MinFr30.mat",0],
          ["Inspected/Inspected/v8_crop-1_MinFr30.mat",1],
          ["Inspected/Inspected/v9_crop-1_MinFr30.mat",0],
          ["Inspected/Inspected/v9_crop-1_MinFr30.mat",1]]

    In += [["Inspected/Inspected/v3_crop-2_MinFr30.mat",2],

            ["Inspected/Inspected/v9_crop-2_MinFr30.mat",0],
          ["Inspected/Inspected/v19_crop-2_MinFr30.mat",0],
          ["Inspected/Inspected/v25_crop-1_MinFr50.mat",0]]

    In += [["/home/jarbona/Downloads/M5_C1_tracks_compiled.mat",0]]

    lw = ["saved_weights/three_layer_trained_on_no_sub_anisentropy0_diffsigma1_delta_sigma_directed3",
          "training_0separation_4_6" ]

    #graph.load_weights("saved_weights/three_bilayer_sub_bis")
    #graph.load_weights("diff_sigma=2.0,delta_sigma_directed=6.,ndim=2,anisentropy=0.1,deltav=1.0,rho_fixed=False,random_rotation=True_2_6")

    #graph.load_weights("diff_sigma=2.0,delta_sigma_directed=6.,ndim=2,anisentropy=0.1,deltav=1.0,rho_fixed=False,random_rotation=True_2_6")
    """

    graph7.load_weights("saved_weights/two_bilayer_without_sub")
    graph7.load_weights("two_attemp_improve_sh_9_66")
    graph7.load_weights("two_attemp_improve_2_54")"""
    #graph7.load_weights("two_attemp_improve_sh_delta6_fix_2_12")
    #graph.load_weights("saved_weights/three_layer_trained_on_no_sub_anisentropy0_diffsigma1_delta_sigma_directed3")
    #graph.load_weights("three_attemp_improve_sh_delta_6_fix_2_6")

    #graph.load_weights("three_attemp_improve_sh_delta_3_20_12")
    #graph.load_weights("three_attemp_improve_sh_delta_0p5_dv_0p1_2_60")
    #graph.load_weights("three_attemp_improve_sh_delta_1_dv_0p1_2_54")
    #graph.load_weights("saved_weights/three_bilayer_sub_bis")
    #graph.load_weights("saved_weights/three_bilayer_sub_bis")
    #graph7.load_weights("two_attemp_improve_sh_delta8_2_6")
    #graph.load_weights("sub_with_p60_noise_7_0")
    #graph.load_weights("sub_with_p60_noise_2_48")
    count = 0
    for filee,traje in In:#[:1]:

        for w in lw[:1]:
            #print w[-10:]
            #graph7.load_weights(w)
            r = process_one_file(filee,graph,sm=0)

            traj,cat,pred_cat = r["results"][traje]

            cat = np.array(cat)
            cat[np.isnan(cat)] = 9
            F = figure()
            ax = F.add_subplot(131)
            plot_label(traj,cat,remove6=9)
            ax = F.add_subplot(132)
            plot_by_class(traj,cat)

            ax = F.add_subplot(133)
            pylab.savefig("traj%i"%count)
            count += 1
            pylab.clf()

            #print pred_cat.shape
            pcat = pred_cat[0]

            bar(np.arange(len(pcat))-0.5,pcat)
            plt.xticks(range(len(pcat)))


In [2]:
from Automated_test import Brownian_Brownian_separation,Brownian_V_separation
import pylab
def generate_report(graph,name="output.pdf",title="trained 0.5 noise",v=1):
    
    r = Brownian_V_separation(graph,sm=0,g4=False,v=v)
    pylab.savefig("fig1.png")
    pylab.clf()
    r = Brownian_V_separation(graph,sm=0,g4=False,noise_level=0.25,v=v)
    pylab.savefig("fig2.png")
    pylab.clf()

    Brownian_Brownian_separation(graph,noise_level=0,sm=0,g4=False,v=v)
    pylab.savefig("fig3.png")
    pylab.clf()

    pylab.savefig("fig4.png")
    pylab.clf()
    
    create_images_report(graph,v=v)

    
    with open("template.html","w") as f:
        f.writelines(template%(title,secondtable))
    
    import pdfkit
    pdfkit.from_file('template.html', name)

/usr/lib/python2.7/dist-packages/sklearn/utils/sparsetools/__init__.py:3: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._min_spanning_tree import minimum_spanning_tree
/usr/lib/python2.7/dist-packages/sklearn/utils/sparsetools/_graph_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._graph_tools import csgraph_to_dense, csgraph_from_dense,\
/usr/lib/python2.7/dist-packages/sklearn/utils/sparsetools/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._traversal import connected_components
/usr/lib/python2.7/dist-packages/sklearn/utils/extmath.py:20: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from ._logistic_sigmoid import _log_logistic_sigmoid
/usr/lib/python2.7/dist-packages/sklearn/utils/extmath.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  from .sparsefuncs_fast impor

In [50]:
#generate_report("prout")